# Agrupamiento de Secuencias
Se muestra cómo realizar el agrupamiento de secuencias de visitas a páginas web. Usaremos el algoritmo K-means adaptado para manejar secuencias, empleando la distancia de Levenshtein como medida de similitud entre secuencias. Este ejemplo es simplificado y asume que las secuencias ya están preprocesadas y codificadas como listas de enteros.

In [2]:
!pip install Levenshtein

  Obtaining dependency information for Levenshtein from https://files.pythonhosted.org/packages/47/19/4528246e25bb79fa8d4adae6640251c613f05eb310d79307d1ac53c7bf28/Levenshtein-0.25.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for rapidfuzz<4.0.0,>=3.8.0 from https://files.pythonhosted.org/packages/04/10/2c0ef45d4ace8dde87cfb91e48fb5c9976f8e01a57eb3230d90b82801dc5/rapidfuzz-3.9.3-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/98.4 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/98.4 kB ? eta -:--:--
   ------------ --------------------------- 30.7/98.4 kB 217.9 kB/s eta 0:00:01
   ---------------- ----------------------- 41.0/98.4 kB 279.3 kB/s eta 0:00:01
   --------------------------------- ------ 81.9/98.4 kB 416.7 kB/s eta 0:00:01
   --------------------------------- ------ 81.9/98.4 kB 416.7 kB/s eta 0:00:01
   ---------------------------------------- 98.4/98.4 kB 352.5 kB/s eta 0:00:00
   -------------

In [3]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
import Levenshtein

### Función de Distancia de Levenshtein:
Esta función calcula la matriz de distancias entre todas las secuencias, utilizando la distancia de Levenshtein.

In [4]:
# Función para calcular la matriz de distancias basada en la distancia de Levenshtein
def levenshtein_distance_matrix(sequences):
    n = len(sequences)
    distances = np.zeros((n, n))
    for i in range(n):
        for j in range(i + 1, n):
            dist = Levenshtein.distance(''.join(map(str, sequences[i])), ''.join(map(str, sequences[j])))
            distances[i, j] = dist
            distances[j, i] = dist
    return distances

In [5]:
# Datos de ejemplo: secuencias de páginas visitadas (cada número representa una página diferente)
sequences = [
    [1, 2, 3, 4],
    [1, 2, 4, 3],
    [1, 3],
    [2, 3, 4],
    [3, 4, 1, 2],
    [1, 2, 3],
    [3, 1, 2, 4],
]

In [6]:
# Encontrar la longitud máxima de las secuencias para rellenar las más cortas
max_len = max(len(seq) for seq in sequences)

In [7]:
print(max_len)

4


### Relleno (Padding) de Secuencias:
Para asegurarnos de que todas las secuencias tengan la misma longitud, rellenamos las secuencias más cortas con un valor distintivo (en este caso, -1).

In [8]:
# Rellenar (padding) las secuencias más cortas con un valor distintivo (por ejemplo, -1)
padded_sequences = [seq + [-1] * (max_len - len(seq)) for seq in sequences]

In [9]:
padded_sequences

[[1, 2, 3, 4],
 [1, 2, 4, 3],
 [1, 3, -1, -1],
 [2, 3, 4, -1],
 [3, 4, 1, 2],
 [1, 2, 3, -1],
 [3, 1, 2, 4]]

### Matriz de Distancias:
Se calcula la matriz de distancias para las secuencias rellenadas.

In [10]:
# Calcular la matriz de distancias personalizada
dist_matrix = levenshtein_distance_matrix(padded_sequences)

In [11]:
dist_matrix

array([[0., 2., 5., 3., 4., 2., 2.],
       [2., 0., 5., 4., 4., 3., 2.],
       [5., 5., 0., 3., 4., 3., 4.],
       [3., 4., 3., 0., 3., 2., 4.],
       [4., 4., 4., 3., 0., 4., 2.],
       [2., 3., 3., 2., 4., 0., 4.],
       [2., 2., 4., 4., 2., 4., 0.]])

### K-means:
Se aplica el algoritmo K-means utilizando la matriz de distancias como entrada para agrupar las secuencias en clústeres.

In [12]:
# Aplicar K-means con la matriz de distancias personalizada
n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(dist_matrix)

C:\Users\Juan\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\Juan\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


### Resultados:
Se imprimen los clústeres asignados a cada secuencia original.

In [13]:
# Mostrar los clústeres asignados a cada secuencia
for idx, cluster in enumerate(clusters):
    print(f'Secuencia {idx + 1}: {sequences[idx]} -> Clúster {cluster}')

Secuencia 1: [1, 2, 3, 4] -> Clúster 2
Secuencia 2: [1, 2, 4, 3] -> Clúster 2
Secuencia 3: [1, 3] -> Clúster 1
Secuencia 4: [2, 3, 4] -> Clúster 1
Secuencia 5: [3, 4, 1, 2] -> Clúster 0
Secuencia 6: [1, 2, 3] -> Clúster 1
Secuencia 7: [3, 1, 2, 4] -> Clúster 0
